# Interpolation

`dolfin.interpolate()` computes the standard (local) nodal interpolant, given by

$$ I_K (f) = \sum_{i = 1}^{n_K} L_i (f) \phi_i $$

for all dofs $L_i$ over cell $K$. This means that `interpolate()` cannot work in Hermite function spaces without extending the interface for `ufc::function` to include differentiation. In particular we need to be able to take partial derivatives along each spatial coordinate (in the notation of the paper $L_{\alpha_i}, 1 \leq i \leq d$) of the function $f$ and use these values as the coefficients for the corresponding shape functions $\phi_{\alpha_i}$ in the representation in the global basis of $V_h$.

This notebook will contain progress in this matter.

## Current status (FEniCS 2017.0.1)

The default implementation (in `ffc/evaluatedof.py:_generate_body()`) of code generation for

```c++
double evaluate_dof(std::size_t i,
                      const ufc::function& f,
                      const double * coordinate_dofs,
                      int cell_orientation,
                      const ufc::cell& c) const final override
```

does the following:

1. Get coordinate for dof `i`.
2. Call `f.evaluate()` on this coordinate.

I.e. it assumes that all dofs are point evaluations! What is the best way of implementing `evaluate_dof()` for derivative evaluations? Do I need to extend the interface of GenericFunctions|whatever to include derivatives? Can I use automatic differentiation in UFL?

In this notebook:

* Nodal interpolation of `Callable`s into Hermite and DKT `FunctionSpace`s works (actually into any space using only `PointEvaluation` and `PointDerivative`). See [utils](utils.py) for the automatic differentiation.
* **FIXME:** $H^1$ norms are [very wrong](#Testing-the-norms). This a problem with `norm()` (Hermite coefficients seem to be ok)

# A hack using AD

In [ ]:
# Boilerplate
from dolfin import *
%matplotlib inline
import matplotlib.pyplot as pl
import autograd.numpy as np
from utils import make_derivatives, ExpressionAD
from FIAT import functional
from ffc import fiatinterface
import nbimporter

def __nbinit__():
    global __all__
    __all__ = ['interpolate_hermite', 'make_constant']

We need to be able to take derivatives. A first attempt might be to [use UFL](#Using-UFL) to compute derivatives of `Expressions` but I can't get this to work. Instead, we use actual python functions and compute their derivatives with `autograd` (see [utils.py](utils.py) for the definition of the decorator `make_derivatives`).

Here is how we can replace the evaluation of an already defined `Expression`:

In [ ]:
V = FunctionSpace(UnitSquareMesh(10, 10), 'Hermite', 3)
f = Expression("x[0]*x[1]*x[1]", degree=3)
# This won't work:
#v = interpolate(f, V)

# Instead define a function which can (automatically) differentiated
@make_derivatives
def fun2d(x, y):
    return x*(y+2)+x**2

# Setup the evaluation on a grid
tt = np.linspace(0, 1, 10)
xx, yy = np.meshgrid(tt, tt)
pp = np.dstack((xx,yy))
points_shape = pp.shape[:-1]
zz = np.zeros(points_shape)

# Evaluate the expression using fun2d.
# The original definition of the expression is ignored!
for i,j in np.ndindex(points_shape):
    zz[i,j] = f.evaluate(pp[i,j], {f: fun2d}, (), None, derivatives=(0,1))

_ = pl.imshow(zz, aspect='auto', cmap='bone')

That was nice, but instead we want to subclass `Expression` to be able to reuse the declared expressions both in regular FEniCS code and below in our `interpolate_hermite()`, which requires taking derivatives (again, see [utils.py](utils.py) for the definition). Here's how to use it:

In [ ]:
# The function we are going to wrap
def f(x, y):
    return x*y

# Usage
e = ExpressionAD(fun=f, degree=3)

# Testing
val = np.zeros(1)
x = np.array([2., 3.])
e.eval(val, x)
print(val)
e.partial(val, x, (1,1))
print(val)
e(x, derivatives=(1,0))

Once we can take derivatives of our `Expressions`, we can interpolate these onto `FunctionSpaces` made of Lagrange and Hermite elements:

In [ ]:
def interpolate_hermite(f:ExpressionAD, V:FunctionSpace) -> Function:
    """ Nodal interpolation for elements using Hermite or Lagrange DOFs.
    
    This is just a hack and requires an actual python function
          which we can evaluate and differentiate.
    
    Arguments
    ---------
        f: Can be one of:
           * ExpressionAD wrapper around a python function of tdim arguments.
             Example:
                 V = FunctionSpace(some2dmesh, 'Hermite', 3)
                 fun = lambda x,y: x*y**2
                 f = ExpressionAD(fun=f, degree=3)
                 u = interpolate_hermite(f, V)

           * Callable with signature (np.ndarray, derivatives=()) where the
             array is of dimension equal to the topological dimension of the mesh.
             Apply the decorator @make_derivatives to a function of n variables
             to use autograd's AD to create the callable.
             Example:

                @make_derivatives
                def fun2d(x, y):
                    return x*y
                u = interpolate_hermite(fun2d, V)
                # Equivalently, with the definition of fun above:
                v = interpolate_hermite(make_derivatives(fun), V)

            Now fun2d() has signature fun2d(np.ndarray, derivatives=())
        V: FunctionSpace using Hermite elements.

    Returns
    -------
        A Function in V.
    """
    warning("WARNING: Nodal interpolation for Hermite dofs. HACK HACK.")
    assert callable(f), "Need a callable (with signature (x, derivatives=())->f(x))"
    
    tdim = V.element().topological_dimension()
    e = fiatinterface.create_element(V.ufl_element())
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    # Note that we cannot use list_hermite_dofs() because we need to iterate over
    # cells to evaluate the partial derivatives in the right order.
    hmask = np.array(list(map(lambda f: isinstance(f, functional.PointDerivative),
                              e.dual_basis())))
    lmask = np.array(list(map(lambda f: isinstance(f, functional.PointEvaluation),
                              e.dual_basis())))
    assert ((hmask | lmask) == True).size == len(e.dual_basis()),\
           "Only Lagrange and Hermite dofs supported."

    #print("hmask = %s" % hmask)
    uv = np.zeros(V.dim())
    dm = V.dofmap()
    dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, tdim))
    if tdim == 1:
        derivatives = [(1,)]*2
    elif tdim == 2:
        derivatives = [(1,0),(0,1)]*3
    elif tdim == 3:
        derivatives = [(1,0,0),(0,1,0),(0,0,1)]*4
    for i in range(V.mesh().num_cells()):
        dofs = dm.cell_dofs(i)
        hermite_dofs = dofs[hmask]
        lagrange_dofs = dofs[lmask]
        for dd, hdof in zip(derivatives, hermite_dofs):
            uv[hdof] = f(dof_coordinates[hdof], derivatives=dd)
            #print("hdof=%d, coords=%s, derivs=%s" % (hdof, dof_coordinates[hdof], dd))
        for ldof in lagrange_dofs:
            uv[ldof] = f(dof_coordinates[ldof], derivatives=())
    u = Function(V)
    u.vector().set_local(uv)

    return u

Here is how to use that:

In [ ]:
V = FunctionSpace(UnitSquareMesh(20, 20), 'DKT', 3)
def fun2d(x, y):
    return x*(y+2)+x**2
f = ExpressionAD(fun=fun2d, degree=3)
u = interpolate_hermite(f, V)
v = project(f, V)
w = Function(V, u.vector() - v.vector())

pl.figure(figsize=(12,4))
pl.subplot(1,3,1)
plot(u, title="Interpolation into H3", cmap='bone')
pl.subplot(1,3,2)
plot(v, title="Projection onto H3", cmap='bone')
pl.subplot(1,3,3)
_ = plot(w, title="Difference (ok?)", cmap='bone')

The difference is very strange:

In [ ]:
wv = u.vector() - v.vector()
print((wv.min(), wv.max()))

xx = np.linspace(0, 1, 20)
for y in xx:
    pl.plot(xx, [w(x, y) for x in xx])
_ = pl.title("$u-v$")

# Constants

Constants are no different and need to be interpolated with `interpolate_hermite()`, since `dolfin.interpolate()` would set all coefficients for the Hermite dofs to the value of the constant. Here is a handy shortcut:

In [ ]:
def make_constant(value:float, V:FunctionSpace):
    """ Returns a constant Function over V.
    
    This only makes sense when V is built with elements using
    Hermite dofs. For other FunctionSpaces this just returns
    a Constant.
    
    Arguments
    ---------
        value: a scalar.
        V: FunctionSpace of elements using Hermite dofs

    Returns
    -------
        A Function in V.
    """
    if V.ufl_element().shortstr().lower()[:3] not in ['her', 'dkt']:
        return Constant(value)

    def fun(x, derivatives=()):
        return value if sum(derivatives) == 0 else 0.

    return interpolate_hermite(fun, V)

# Testing the norms

** Warning:**

Computing the norm using `dolfin.norm()` is probably an error since *all* coefficients are used, including the Hermite ones, so the value obtained is not an approximation of the real $L^2$ norm of the function.

In [ ]:
from IPython.display import Math
print("The following norms are wrong!")
Math("||u_h - u_p||_{L^2} = %.4f, \  ||u_h - u_p||_{L^{\infty}} = %.4f" % 
     (norm(w), norm(w.vector(), norm_type='linf')))

Now compare that to the computation using only the values at the vertices (i.e. the Lagrange dofs)

In [ ]:
u_vert = u.compute_vertex_values()
v_vert = v.compute_vertex_values()
print("%.2f%% of vertices where u and v disagree more than 1e-6. However:" % 
      (100.*np.nonzero(np.abs(u_vert-v_vert) > 1e-6)[0].size / V.dim()))
Math("\\frac{||u_h - u_p||_{L^2}}{\\text{max}(||u_h||_{L^2}, ||u_p||_{L^2})} = %.4f, \  "
     "\\frac{||u_h - u_p||_{L^{\infty}}}{\\text{max}(||u_h||_{L^{\infty}},||u_p||_{L^{\infty}})} = %.4f" % 
     (np.linalg.norm(u_vert-v_vert)/np.maximum(np.linalg.norm(u_vert), np.linalg.norm(v_vert)),
      np.linalg.norm(u_vert-v_vert, ord=np.inf)/np.max(np.maximum(u_vert, v_vert))))

Yet another example. Notice how wrong the computation of the $H^1$ norm is:

In [ ]:
def ftest(x, y):
    return x**2+y**2
f = ExpressionAD(fun=ftest, degree=3)
V = FunctionSpace(UnitSquareMesh(20, 20), 'DKT', 3)
w1 = interpolate_hermite(f, V)
w2 = project(f, V)
print("           w1\t\tw2\t\texact " % ())
print("L2       : %f, %f, %f" % (norm(w1, 'L2'), norm(w2, 'L2'), np.sqrt(28/45.)))
print("H10 norms: %f, %f, %f" % (norm(w1, 'H10'), norm(w2, 'H10'), np.sqrt(8/3.)))
print("H1 norms : %f, %f, %f" % (norm(w1, 'H1'), norm(w2, 'H1'), np.sqrt(8/3. + 28/45.)))
#Math("\\text{Exact: }||f||_{L^2} = %f,\ ||f||_{H^1_0} = %f,\ ||f||_{H^1} = %f" %
#      (np.sqrt(28/45.), np.sqrt(8/3.), np.sqrt(8/3. + 28/45.)))

# Attempts using UFL

A couple of tests trying to use AD on expressions in order to implement an `interpolate()` 

In [ ]:
from ufl.algorithms.apply_derivatives import apply_derivatives

V = FunctionSpace(UnitIntervalMesh(2), 'Hermite', 3)
x = SpatialCoordinate(V.mesh())
f = x[0]**2
str(f)
#replace(f, {x:0.})  # Doesn't work
dfdx = apply_derivatives(diff(f, x))
#replace(dfdx, {x: 0.})  # Doesn't work

## A test

The first thing to check once `evaluate_dof()` has been implemented for Hermite elements is a constant:

In [ ]:
u = interpolate(Constant(2.0), V)
_ = plot(u, cmap='bone')